In [19]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [20]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_true,urllc_req,urllc_true,miot_req,miot_true,mmtc_req,mmtc_true,voice_req,voice_true
round,,,,,,,,,,
0,13405922974,13030020496,198602474,198602474,102163362,102163362,54668699,54668699,190728724,190728724
1,14891822876,13832043931,95467765,95467765,105346529,105346529,67839887,67839887,268679119,268679119
2,13959637690,13839867790,103193515,103193515,46819034,46819034,59392887,59392887,314980021,314980021
3,31850806940,14948808304,434672547,434672547,336038788,336038788,221439466,221439466,509947726,509947726
4,26177130335,15232227585,51288544,51288544,37400967,37400967,25563860,25563860,412911391,412911391


In [21]:
# Parametes 
TRAIN_SPLIT = 64000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 32 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 80000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 2000 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 500 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 20
bs_capacity = 20000000000


In [22]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity


In [23]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [24]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [25]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 2000 steps, validate for 500 steps
Epoch 1/20
2000/2000 [==============================] - 13s 6ms/step - loss: 0.0257 - val_loss: 0.0237
Epoch 2/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0243 - val_loss: 0.0222
Epoch 3/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0241 - val_loss: 0.0225
Epoch 4/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0240 - val_loss: 0.0236
Epoch 5/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0239 - val_loss: 0.0227
Epoch 6/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0238 - val_loss: 0.0224
Epoch 7/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0238 - val_loss: 0.0248
Epoch 8/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0237 - val_loss: 0.0231
Epoch 9/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0237 - val_loss: 0.0218
Epoch 10/20
2000/2000 [=====

In [26]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[28117743919, 554516117, 283582751, 309281138, 510198997]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[2]) # because I took the input of the first values in x_train for test


[0.4033091  0.005623   0.0038313  0.00235694 0.0053687 ]
[0.31420964 0.00469123 0.00282344 0.00143392 0.0040093 ]
[0.8540199  0.00686314 0.00492667 0.00139339 0.01798228]
[0.76161138 0.00256443 0.00187005 0.00127819 0.02064557]


In [27]:
# Save trained model
model.save("model.h5")
